# // Getting Individual Player Data
___
The first step in getting this project underway is going to be getting **massive** amounts of NFL data from the web. I will be working in this notebook to "show my work" and for others to learn how to if they're curious. Ultimately, I'll also turn it into a regular `.py` Python script that you can run if you're so inclined.

For that, we're going to rely on the `requests` and `BeautifulSoup` libraries to glean information from:

- [FantasyPros](https://www.fantasypros.com/)
- [Pro Football Reference](https://www.pro-football-reference.com/)
- [FFToday](http://www.fftoday.com/stats/)
- [The Football Database](https://www.footballdb.com/fantasy-football/index.html)

**Editor's Note:** After researching several different options for quite some time and attempting to use many of them, I will be going with **FantasyPros** for reasons outlined below.

In [42]:
# Importing our necessary libraries
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
%matplotlib inline

## FantasyPros
I'm choosing to go with this FantasyPros for the following reasons:
- You don't have to register to access the data
- Separately, I am currently a user and like their site overall
- Has available .5 PPR scoring data (FanDuel does .5 PPR so DFS appeal, and it will be most applicable for most leagues)
- The Football Database seemed like it was missing quite a few players from each week
- FFToday scoring was inconsistent in many places for .5 PPR scoring

#### Following the FanDuel vein, since Daily Fantasy Sports (DFS) is where the biggest value is going to be coming from, I will be scraping data for the following positions and referring to them from here on out by the name in parentheses:
1. Quarterback (QB)
2. Running Back (RB)
3. Wide Receiver (WR)
4. Tight End (TE)
5. Defense / Special Teams (DST)


> - For a more detailed breakdown on positional scoring, please reference FanDuel's scoring and rules reference [here](https://www.fanduel.com/rules).
- For a quick reference on what each column means, check out ESPN's stat reference [here](http://www.espn.com/nfl/news/story?id=2128923).

___
### Scraping for our QBs
Going to walk through the specific steps for this first position then blast through the others.

• Let's make some empty lists to throw all of our data into:

In [43]:
player = []
pass_comp = []
pass_att = []
pass_pct = []
pass_yds = []
yds_per_att = []
pass_TD = []
pass_INT = []
sacks_taken = []
rush_att = []
rush_yds = []
rush_TD = []
fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

In [44]:
# Make sure to incoporate this into the scrape!
week = []
year = []

• And a list of those lists to make `dataframe` creation nice and easy:

In [45]:
qb_stats_lists = [player, pass_comp, pass_att, pass_pct, pass_yds, 
                  yds_per_att, pass_TD, pass_INT, sacks_taken, 
                  rush_att, rush_yds, rush_TD, fumbles_lost, 
                  active, fpoints, fpoints_g, own_pct]



• Doing the actual scrape by going through each week of the season (via each URL iteration), grabbing the right `table`, iterating through each `row` and then `cell`, and putting that data into the appropriate `lists`.

In [46]:
# First let's add a quick little variable for our current week in the 2018 NFL season
res = requests.get('https://www.fantasypros.com/nfl/')
soup = BeautifulSoup(res.content, 'lxml')
current_week = [int(s) for s in soup.find_all('span', {'class': 'item-title'})[8].text.split() if s.isdigit()][0]




In [47]:
# res = requests.get('https://www.fantasypros.com/nfl/')
# soup = BeautifulSoup(res.content, 'lxml')
# # print(soup.find_all('span', {'class': 'item-title'})[8].text)
# [int(s) for s in soup.find_all('span', {'class': 'item-title'})[8].text.split() if s.isdigit()][0]

In [48]:
for week_number in range(1,current_week):

    res = requests.get('https://www.fantasypros.com/nfl/stats/qb.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')

    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(qb_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    # And adding a 1 second delay 
    sleep(1)

• Let's make an empty `dataframe` and then fill it using our list of lists:

In [49]:
qbdf = pd.DataFrame(columns= ['player', 'pass_comp', 'pass_att', 'pass_pct',
                              'pass_yds', 'yds_per_att', 'pass_TD', 'pass_INT', 
                              'sacks_taken', 'rush_att', 'rush_yds', 'rush_TD', 
                              'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
qbdf

,player,pass_comp,pass_att,pass_pct,pass_yds,yds_per_att,pass_TD,pass_INT,sacks_taken,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct


In [50]:
for index, column in enumerate(qbdf.columns):
    qbdf[column] = qb_stats_lists[index]

• Let's add on our `week` and `year` information as well as a separate `team` column from our `player` column (to later be used to bring in opponent data).

In [51]:
# Week
qbdf['week'] = week
qbdf['week'] = qbdf.week.astype(str)

In [52]:
team = []
for individual in qbdf['player']:
    team.append(re.findall('\(([^\)]+)\)', individual)[0])
qbdf['team'] = team

• And let's just clean up that `player` name a bit too with another `regex`:

In [53]:
qbdf['player'] = [ind[0] for ind in [re.findall('^.*?(?=\s\()', quarterback) for quarterback in qbdf['player']]]

• While we're at the cleanup, we're going to need to do some with the `own_pct`:

In [54]:
qbdf['own_pct'] = qbdf.own_pct.apply(lambda x: x.strip('%'))
qbdf.own_pct.replace(to_replace='', value='0.0', inplace=True)

• Since most if not all of these values were actually read in as `strings`, we'll need to convert them to numeric columns (with the exceptions of `player`, `team`, and `week`.

In [55]:
for x in [col for col in qbdf.columns if col not in ['player','week', 'team', 'year']]:
    qbdf[x] = qbdf[x].astype(float)

**Since we're going to likely be wanting to clean up our other dataframes in the same way, let's make a function to clean up our future code:**

In [56]:
def table_cleaner(df):
    # Adding 'week' as a string
    df['week'] = week
    df['week'] = df.week.astype(str)
    
    # Cleaning 'team'
    team = []
    for individual in df['player']:
        team.append(re.findall('\(([^\)]+)\)', individual)[0])
    df['team'] = team
    
    # Removing team from 'player'
    df['player'] = [ind[0] for ind in [re.findall('^.*?(?=\s\()', person) for person in df['player']]]    
    
    # Converting 'own_pct' to float
    df['own_pct'] = df.own_pct.apply(lambda x: x.strip('%'))
    df.own_pct.replace(to_replace='', value='0.0', inplace=True)    
    
    # Converting everything else to float where appropriate
    for x in [col for col in df.columns if col not in ['player','week', 'team', 'year']]:
        df[x] = df[x].astype(float)
    
    return df

In [57]:
# Checking to see if we'll be able to manipulate it how we want
qbdf.sort_values('fpoints', ascending=False).head(10)

,player,pass_comp,pass_att,pass_pct,pass_yds,yds_per_att,pass_TD,pass_INT,sacks_taken,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
473,Mitch Trubisky,19.0,26.0,73.1,354.0,13.6,6.0,0.0,1.0,3.0,53.0,0.0,0.0,1.0,43.5,43.5,74.1,4,CHI
32,Ryan Fitzpatrick,21.0,28.0,75.0,417.0,14.9,4.0,0.0,0.0,12.0,36.0,1.0,0.0,1.0,42.3,42.3,11.0,1,TB
269,Drew Brees,39.0,49.0,79.6,396.0,8.1,3.0,0.0,1.0,3.0,7.0,2.0,0.0,1.0,40.5,40.5,99.2,3,NO
281,Matt Ryan,26.0,35.0,74.3,374.0,10.7,5.0,0.0,3.0,4.0,12.0,0.0,0.0,1.0,40.2,40.2,94.9,3,ATL
461,Jared Goff,26.0,33.0,78.8,465.0,14.1,5.0,0.0,1.0,2.0,7.0,0.0,0.0,1.0,39.3,39.3,96.2,4,LAR
129,Ben Roethlisberger,39.0,60.0,65.0,452.0,7.5,3.0,0.0,1.0,2.0,9.0,1.0,0.0,1.0,39.0,39.0,96.1,2,PIT
225,Patrick Mahomes,23.0,28.0,82.1,326.0,11.6,6.0,0.0,1.0,5.0,18.0,0.0,0.0,1.0,38.8,38.8,99.2,2,KC
1227,Mitch Trubisky,23.0,30.0,76.7,355.0,11.8,3.0,0.0,1.0,3.0,18.0,1.0,0.0,1.0,36.0,36.0,74.1,10,CHI
1355,Patrick Mahomes,33.0,46.0,71.7,478.0,10.4,6.0,3.0,3.0,6.0,28.0,0.0,2.0,1.0,35.9,35.9,99.2,11,KC
1135,Ben Roethlisberger,22.0,25.0,88.0,328.0,13.1,5.0,0.0,1.0,3.0,27.0,0.0,0.0,1.0,35.8,35.8,96.1,10,PIT


In [58]:
qbdf.week.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15'], dtype=object)

Through Week 5, no player has posted 2 of the season's Top 10 QB performances — yay parity!

In [59]:
qbdf.groupby('player')['fpoints'].mean().sort_values(ascending=False)[0:10]

player
Patrick Mahomes       24.766667
Matt Ryan             20.413333
Ben Roethlisberger    19.873333
Drew Brees            19.160000
Andrew Luck           18.860000
Cam Newton            18.840000
Philip Rivers         18.306667
Deshaun Watson        18.226667
Jared Goff            17.940000
Aaron Rodgers         17.906667
Name: fpoints, dtype: float64

Through Week 5, we see some familiar suspects in the Top 10 points per game, but also some young guns like Mahomes, Goff, Watson, Cousins, and a resurgence for Luck.

• Now let's dump everything we have so far into a `csv` file that we can save to our current working directory:

In [60]:
qbdf.to_csv('./data/qb_stats_2018.csv')

___
### Running Backs
(Note that the tables are structured significantly differently for each position otherwise we would be able to use one nice and clean single scrape).

In [61]:
player = []

rush_att = []
rush_yds = []
rush_ypc = []
long = []
over_20 = []
rush_TD = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
rec_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [62]:
rb_stats_lists = [player,rush_att,rush_yds,rush_ypc,long,over_20,rush_TD,
                  receptions,targets,rec_yds,rec_ypr,rec_TD,
                  fumbles_lost,active,fpoints,fpoints_g,own_pct]

for week_number in range(1,current_week):
    res = requests.get('https://www.fantasypros.com/nfl/stats/rb.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(rb_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

rbdf = pd.DataFrame(columns= ['player','rush_att','rush_yds','rush_ypc','long','over_20','rush_TD',
                  'receptions','targets','rec_yds','rec_ypr','rec_TD',
                  'fumbles_lost','active','fpoints','fpoints_g','own_pct'])
for index, column in enumerate(rbdf.columns):
    rbdf[column] = rb_stats_lists[index]

rbdf = table_cleaner(rbdf)

In [63]:
rbdf.to_csv('./data/rb_stats_2018.csv')

In [64]:
# Little spot check to make sure we're Gucci
rbdf.loc[rbdf['fpoints'] == rbdf.fpoints.max(), :]

,player,rush_att,rush_yds,rush_ypc,long,over_20,rush_TD,receptions,targets,rec_yds,rec_ypr,rec_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
3753,Derrick Henry,17.0,238.0,14.0,99.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,47.8,47.8,86.5,14,TEN


___
### Wide Receivers

In [65]:
player = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
long = []
over_20 = []
rec_TD = []

rush_att = []
rush_yds = []
rush_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [66]:
wr_stats_lists = [player, receptions, targets, rec_yds, rec_ypr, long, over_20, rec_TD,
                  rush_att, rush_yds, rush_TD,
                  fumbles_lost, active, fpoints, fpoints_g, own_pct]

for week_number in range(1,current_week):
    res = requests.get('https://www.fantasypros.com/nfl/stats/wr.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(wr_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

wrdf = pd.DataFrame(columns= ['player', 'receptions', 'targets', 'rec_yds', 'rec_ypr', 'long', 'over_20', 'rec_TD',
                  'rush_att', 'rush_yds', 'rush_TD',
                  'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
for index, column in enumerate(wrdf.columns):
    wrdf[column] = wr_stats_lists[index]

wrdf = table_cleaner(wrdf)

In [67]:
wrdf.to_csv('./data/wr_stats_2018.csv')

In [68]:
# Andddd our usual sanity check
wrdf.loc[wrdf['fpoints'] > 30, :]

,player,receptions,targets,rec_yds,rec_ypr,long,over_20,rec_TD,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
219,Tyreek Hill,7.0,8.0,169.0,24.1,58.0,5.0,2.0,2.0,4.0,0.0,0.0,1.0,32.8,32.8,100.0,1,KC
563,Stefon Diggs,9.0,13.0,128.0,14.2,75.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,31.4,31.4,99.8,2,MIN
1089,Calvin Ridley,7.0,8.0,146.0,20.9,75.0,1.0,3.0,1.0,9.0,0.0,0.0,1.0,37.0,37.0,84.0,3,ATL
1409,Cooper Kupp,9.0,11.0,162.0,18.0,70.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,32.7,32.7,18.9,4,LAR
2065,Davante Adams,10.0,16.0,132.0,13.2,38.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,30.2,30.2,99.9,6,GB
2146,Tyreek Hill,7.0,12.0,142.0,20.3,75.0,2.0,3.0,1.0,0.0,0.0,0.0,1.0,35.7,35.7,100.0,6,KC
3271,Michael Thomas,12.0,15.0,211.0,17.6,72.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,33.1,33.1,100.0,9,NO
3940,T.Y. Hilton,9.0,9.0,155.0,17.2,68.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,32.0,32.0,98.9,11,IND
4054,Tyreek Hill,10.0,14.0,215.0,21.5,73.0,5.0,2.0,0.0,0.0,0.0,0.0,1.0,38.5,38.5,100.0,11,KC
4379,Amari Cooper,8.0,9.0,180.0,22.5,90.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,34.0,34.0,98.6,12,DAL


___
### Tight Ends

In [69]:
player = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
long = []
over_20 = []
rec_TD = []

rush_att = []
rush_yds = []
rush_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [ ]:
te_stats_lists = [player, receptions, targets, rec_yds, rec_ypr, long, over_20, rec_TD,
                  rush_att, rush_yds, rush_TD,
                  fumbles_lost, active, fpoints, fpoints_g, own_pct]

for week_number in range(1,current_week):
    res = requests.get('https://www.fantasypros.com/nfl/stats/te.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(te_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

tedf = pd.DataFrame(columns= ['player', 'receptions', 'targets', 'rec_yds', 'rec_ypr', 'long', 'over_20', 'rec_TD',
                  'rush_att', 'rush_yds', 'rush_TD',
                  'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
for index, column in enumerate(tedf.columns):
    tedf[column] = te_stats_lists[index]

tedf = table_cleaner(tedf)

In [ ]:
tedf.to_csv('./data/te_stats_2018.csv')

In [ ]:
# Chiggity check it
tedf.loc[tedf['fpoints'] > 25, :]

___
### Defense / Special Teams

In [ ]:
player = []

sacks = []
INTs = []
fum_rec = []
force_fum = []
def_td = []
safeties = []
spc_TD = []

active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [ ]:
dst_stats_lists = [player, sacks, INTs, fum_rec, force_fum, 
                   def_td, safeties, spc_TD, 
                   active, fpoints, fpoints_g, own_pct]

for week_number in range(1,current_week):
    res = requests.get('https://www.fantasypros.com/nfl/stats/dst.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(dst_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

dstdf = pd.DataFrame(columns= ['player', 'sacks', 'INTs', 'fum_rec', 'force_fum', 
                   'def_td', 'safeties', 'spc_TD', 
                   'active', 'fpoints', 'fpoints_g', 'own_pct'])
for index, column in enumerate(dstdf.columns):
    dstdf[column] = dst_stats_lists[index]

dstdf = table_cleaner(dstdf)

In [ ]:
dstdf.to_csv('./data/dst_stats_2018.csv')

In [ ]:
dstdf.loc[dstdf.fpoints >= 10, :].sort_values('fpoints', ascending = False)

___
# // Wrap-up
___
In this notebook, we brought in game-level stats for individual positions through the current week of the 2018 season and saved them to our local machine.

Overall, we need to bring in:
- ~~QBs~~
- ~~RBs~~
- ~~WRs~~
- ~~TEs~~
- ~~DSTs~~
- Opponents (including game location)
- Defensive stats
- Team play-calling
- Depth charts?
- Weather